<a href="https://colab.research.google.com/github/marreapato/Forecasting_Paper_XAI/blob/main/Times_cv/Tcrossvalidation_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import numpy as np
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Flatten
import math
# Assuming you have loaded your time series data from Yahoo Finance
# For demonstration purposes, let's create a dummy time series
# Replace this with loading your actual data
# bvsp_data = load_bvsp_data_from_yahoo()

# Example with dummy data
np.random.seed(42)
dates = pd.date_range(start="2007-01-01", end="2023-10-17", freq="D")
bvsp_data = pd.DataFrame({"Close": np.random.rand(len(dates))}, index=dates)

# Feature scaling
scaler = MinMaxScaler()
bvsp_data_scaled = scaler.fit_transform(bvsp_data)

# Assuming you have a time series array and corresponding labels
X = bvsp_data_scaled[:-1]
y = bvsp_data_scaled[1:]

# Function to create LSTM model
def create_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(units=128, activation='relu', input_shape=input_shape))
    model.add(Flatten())
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model



In [29]:
# Time series cross-validation using TimeSeriesSplit
n_splits = 30  # You can adjust the number of splits
tscv = TimeSeriesSplit(n_splits=n_splits)
tscv


TimeSeriesSplit(gap=0, max_train_size=None, n_splits=30, test_size=None)

In [30]:
resultslstm = []
for train_index, test_index in tscv.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Reshape the data for LSTM input
    X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
    X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

    # Create and train the LSTM model
    input_shape = (X_train.shape[1], X_train.shape[2])
    model = create_lstm_model(input_shape)
    model.fit(X_train, y_train, epochs=300, batch_size=32, verbose=0)

    # Evaluate the model on the test set
    mse = model.evaluate(X_test, y_test, verbose=0)
    resultslstm.append(math.sqrt(mse))
    print(f'Mean Squared Error for this fold: {math.sqrt(mse)}')

Mean Squared Error for this fold: 0.2950019980217465
Mean Squared Error for this fold: 0.3077665057408165
Mean Squared Error for this fold: 0.28041623295943646
Mean Squared Error for this fold: 0.2836273961560715
Mean Squared Error for this fold: 0.3074803123529455
Mean Squared Error for this fold: 0.2924808381194519
Mean Squared Error for this fold: 0.28843703777239
Mean Squared Error for this fold: 0.2944880952258539
Mean Squared Error for this fold: 0.2811626722605707
Mean Squared Error for this fold: 0.2886059996891999
Mean Squared Error for this fold: 0.2858505679700252
Mean Squared Error for this fold: 0.2915831159517456
Mean Squared Error for this fold: 0.29306886869601706
Mean Squared Error for this fold: 0.29714548186969403
Mean Squared Error for this fold: 0.28703469323681996
Mean Squared Error for this fold: 0.27234089165400616
Mean Squared Error for this fold: 0.29669482636391176
Mean Squared Error for this fold: 0.2869684431399882
Mean Squared Error for this fold: 0.291596

In [31]:
resultslstm

[0.2950019980217465,
 0.3077665057408165,
 0.28041623295943646,
 0.2836273961560715,
 0.3074803123529455,
 0.2924808381194519,
 0.28843703777239,
 0.2944880952258539,
 0.2811626722605707,
 0.2886059996891999,
 0.2858505679700252,
 0.2915831159517456,
 0.29306886869601706,
 0.29714548186969403,
 0.28703469323681996,
 0.27234089165400616,
 0.29669482636391176,
 0.2869684431399882,
 0.2915969137956947,
 0.28836255774838254,
 0.2775069280567177,
 0.2822954166463575,
 0.2888160892503768,
 0.3005029579919908,
 0.27801007832553465,
 0.29851449452565276,
 0.2722205465426307,
 0.29470405366262925,
 0.29047770356593633,
 0.29456473193101096]

In [32]:
np.mean(resultslstm)

0.28959088164078683